# Technical Factor - Detrended Price Oscillator (DPO)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        close - m_lag(m_avg(close, 12), 7) AS DPO,
        m_avg(DPO, 12) AS MADPO,
        DPO - MADPO AS DPO_OSC,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(DPO > 0 AND m_lag(DPO, 1) <= 0, 1, 0) AS TRBY1,
        IF(DPO < 0 AND m_lag(DPO, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(DPO > MADPO AND m_lag(DPO, 1) <= m_lag(MADPO, 1), 1, 0) AS MTBY1,
        IF(DPO < MADPO AND m_lag(DPO, 1) >= m_lag(MADPO, 1), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(DPO < 0 AND DPO_OSC > m_lag(DPO_OSC, 1) AND DPO_OSC < 0, 1, 0) AS RVBY1,
        IF(DPO > 0 AND DPO_OSC < m_lag(DPO_OSC, 1) AND DPO_OSC > 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(DPO, 1) <= 0 AND DPO > 0 AND (DPO - m_lag(DPO, 1)) > 0, 1, 0) AS BKBY1,
        IF(m_lag(DPO, 1) >= 0 AND DPO < 0 AND (DPO - m_lag(DPO, 1)) < 0, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        DPO,
        MADPO,
        DPO_OSC,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,DPO,MADPO,DPO_OSC,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-20,600519.SH,294.040219,5.194149,288.846069,0,0,0,0,0,0,0,0
1,2025-02-21,600519.SH,417.769812,74.457582,343.312230,0,0,0,0,0,0,0,0
2,2025-02-24,600519.SH,349.927953,131.184233,218.743721,0,0,0,0,0,1,0,0
3,2025-02-25,600519.SH,141.060018,154.396118,-13.336100,0,0,0,1,0,0,0,0
4,2025-02-26,600519.SH,169.231292,183.060107,-13.828815,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,2025-12-25,600519.SH,130.969655,-114.489598,245.459253,1,0,0,0,0,0,1,0
210,2025-12-26,600519.SH,140.944512,-64.312553,205.257065,0,0,0,0,0,1,0,0
211,2025-12-29,600519.SH,41.963913,-29.930991,71.894904,0,0,0,0,0,1,0,0
212,2025-12-30,600519.SH,-65.163787,-12.887270,-52.276516,0,1,0,1,0,0,0,1
